# IBM Capstone Assignment

This notebook will be used to execute the Capstone Project - The Battle of Neighborhoods 

### PART ONE : Data Cleaning

In [1]:
!pip install bs4

In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Webscrapping to extract html table using BeautifulSoup

In [3]:
url=('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,'html5lib')
toronto_table = soup.find_all('table', class_="wikitable sortable")

Converting table to dataframe using pandas

In [4]:
df = pd.read_html(str(toronto_table))[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [5]:
print('Dataset size',df.shape)

Dataset size (180, 3)


Cleaning table by removing all rows containing 'Not assigned'

In [6]:
df2 = df[['Postal Code','Borough','Neighbourhood']] 
df2 = df2.set_index('Borough', drop=False)
df2 = df2.drop('Not assigned', axis=0)
df2 = df2.reset_index(drop=True)
df2.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
print('New dataset size',df2.shape)

New dataset size (103, 3)


### PART TWO : Geolocation - Longitude and Latitude

Importing Libraries

In [29]:
!conda install -c conda-forge geopy 

Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |           1_llvm           5 KB  conda-forge
    _py-xgboost-mutex-2.0      |            cpu_0           8 KB  conda-forge
    _pytorch_select-0.2        |            gpu_0           2 KB
    absl-py-0.12.0             |     pyhd8ed1ab_0          96 KB  conda-forge
    aiohttp-3.7.4              |   py37h5e8e339_0         

more-itertools-8.7.0 | 39 KB     | ##################################### | 100% 
backports-1.0        | 4 KB      | ##################################### | 100% 
tqdm-4.59.0          | 77 KB     | ##################################### | 100% 
patsy-0.5.1          | 187 KB    | ##################################### | 100% 
chardet-4.0.0        | 204 KB    | ##################################### | 100% 
icu-68.1             | 13.0 MB   | ##################################### | 100% 
libgcc-ng-9.3.0      | 7.8 MB    | ##################################### | 100% 
markdown-3.3.4       | 67 KB     | ##################################### | 100% 
zipp-3.4.1           | 11 KB     | ##################################### | 100% 
mysql-common-8.0.23  | 1.5 MB    | ##################################### | 100% 
tensorflow-estimator | 645 KB    | ##################################### | 100% 
snowballstemmer-2.1. | 57 KB     | ##################################### | 100% 
imageio-2.9.0        | 3.1 M

jpeg-9d              | 264 KB    | ##################################### | 100% 
glib-2.66.7          | 443 KB    | ##################################### | 100% 
zstd-1.4.9           | 431 KB    | ##################################### | 100% 
unixodbc-2.3.9       | 293 KB    | ##################################### | 100% 
future-0.18.2        | 714 KB    | ##################################### | 100% 
sip-4.19.25          | 295 KB    | ##################################### | 100% 
werkzeug-1.0.1       | 239 KB    | ##################################### | 100% 
prompt-toolkit-3.0.1 | 244 KB    | ##################################### | 100% 
brunsli-0.1          | 200 KB    | ##################################### | 100% 
ibm-wsrt-py37main-ma | 2 KB      | ##################################### | 100% 
gast-0.4.0           | 12 KB     | ##################################### | 100% 
gst-plugins-base-1.1 | 2.5 MB    | ##################################### | 100% 
tabulate-0.8.9       | 26 KB

matplotlib-3.3.4     | 6 KB      | ##################################### | 100% 
requests-2.25.1      | 51 KB     | ##################################### | 100% 
async-timeout-3.0.1  | 11 KB     | ##################################### | 100% 
parso-0.8.1          | 67 KB     | ##################################### | 100% 
pytest-6.2.2         | 430 KB    | ##################################### | 100% 
libblas-3.9.0        | 12 KB     | ##################################### | 100% 
fastcache-1.1.0      | 30 KB     | ##################################### | 100% 
pytables-3.6.1       | 1.5 MB    | ##################################### | 100% 
libclang-11.1.0      | 19.2 MB   | ##################################### | 100% 
secretstorage-3.3.1  | 24 KB     | ##################################### | 100% 
pyqtchart-5.12       | 256 KB    | ##################################### | 100% 
pyasn1-0.4.8         | 53 KB     | ##################################### | 100% 
libtiff-4.2.0        | 633 K

done


In [30]:
! pip install folium==0.5.0

     |████████████████████████████████| 79 kB 6.3 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=ecafcab5fb8f4d673743b2f38ee8c00879750702c90907b1b606f5c8b6dd98c1
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium


In [31]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Importing Geospatial Data using pandas

In [ ]:
df3 = pd.read_csv('https://cocl.us/Geospatial_data')
df3.head()

Merging location and geospatial dataframes using left join function to create one dataframe

In [21]:
df4 = pd.merge(left=df2, right=df3, how='left', left_on='Postal Code', right_on='Postal Code')
df4.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [27]:
df4['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Toronto/York', 'Mississauga'], dtype=object)

### PART THREE : Clustering and Exploration of Neighbourhoods

Using geopy library to get the latitude and longitude values of New York City.

In [32]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [54]:
neighborhoods = df4.drop(['Postal Code'], axis=1, inplace=False)
neighborhoods.head()

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.753259,-79.329656
1,North York,Victoria Village,43.725882,-79.315572
2,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [58]:
map_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(neighborhoods['Borough'], neighborhoods['Neighbourhood'],neighborhoods['Latitude'], neighborhoods['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [85]:
toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,Downtown Toronto,St. James Town,43.651494,-79.375418
4,East Toronto,The Beaches,43.676357,-79.293031


1. Downtown Toronto

In [74]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="dt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


In [70]:
downtown_toronto_data = neighborhoods[neighborhoods['Borough']==('Downtown Toronto')].reset_index(drop=True)

#create map using latitude and longitude values
map_downtown_toronto = folium.Map(location=[43.6541737, -79.38081164513409], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(downtown_toronto_data['Borough'], downtown_toronto_data['Neighbourhood'],downtown_toronto_data['Latitude'], downtown_toronto_data['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown_toronto)  
    
map_downtown_toronto

2. East Toronto

In [73]:
address = 'East Toronto'

geolocator = Nominatim(user_agent="et_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of East Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of East Toronto are 43.6261221, -79.3950351.


In [72]:
east_toronto_data = neighborhoods[neighborhoods['Borough']==('East Toronto')].reset_index(drop=True)

#create map using latitude and longitude values
map_east_toronto = folium.Map(location=[43.6261221, -79.3950351], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(east_toronto_data['Borough'], east_toronto_data['Neighbourhood'],east_toronto_data['Latitude'], east_toronto_data['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_east_toronto)  
    
map_east_toronto

3. West Toronto

In [75]:
address = 'West Toronto'

geolocator = Nominatim(user_agent="wt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of West Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of West Toronto are 43.6534817, -79.3839347.


In [76]:
west_toronto_data = neighborhoods[neighborhoods['Borough']==('West Toronto')].reset_index(drop=True)

#create map using latitude and longitude values
map_west_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(west_toronto_data['Borough'], west_toronto_data['Neighbourhood'],west_toronto_data['Latitude'], west_toronto_data['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_west_toronto)  
    
map_west_toronto

4. Central Toronto

In [90]:
address = 'Central Toronto'

geolocator = Nominatim(user_agent="ct_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6534817, -79.3839347.


In [93]:
central_toronto_data = neighborhoods[neighborhoods['Borough']==('Central Toronto')].reset_index(drop=True)

#create map of using latitude and longitude values
map_central_toronto = folium.Map(location=[43.6534817, -79.3839347], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(central_toronto_data['Borough'], central_toronto_data['Neighbourhood'],central_toronto_data['Latitude'], central_toronto_data['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_central_toronto)  
    
map_central_toronto

5. Toronto/York

In [92]:
address = 'Toronto/York'

geolocator = Nominatim(user_agent="tyt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Toronto are 43.6896191, -79.479188.


In [94]:
toronto_york_data = neighborhoods[neighborhoods['Borough']==('Toronto/York')].reset_index(drop=True)

#create map of using latitude and longitude values
map_toronto_york = folium.Map(location=[43.6896191, -79.479188], zoom_start=10)

# add markers to map
for borough, neighborhood, lat, lng in zip(toronto_york_data['Borough'], toronto_york_data['Neighbourhood'],toronto_york_data['Latitude'], toronto_york_data['Longitude']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_york)  
    
map_toronto_york